> https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqsptt94yl

Un  sitio  de  E books  tiene  información  sobre  los  reviews  que  los  usuarios hacen  de  sus  libros  en  un  DataFrame  con  formato  (user_id,  book_id,  rating, timestamp). 

Por otro lado tenemos información en otro DataFrame que bajamos de  GoodReads:  (book_id,  book_name,  avg_rating).  Podemos  suponer  que  los Ids de los libros son compatibles. 

Se pide usar Python Pandas para:

1. Obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks. (Para este punto se puede ignorar el segundo DataFrame).
2. Obtener un DataFrame que indique qué libros tienen una diferencia de rating promedio mayor al 20% entre el sitio de Ebooks y GoodReads.

---

In [87]:
# Set-up y vista rápida de las dos bases de datos truchas
import pandas as pd

ebooks_df = pd.read_csv('data/2017C1_1_ebooks.csv')
display(ebooks_df)

goodreads_df = pd.read_csv('data/2017C1_1_goodreads.csv')
display(goodreads_df)

,user_id,book_id,rating,timestamp
0,1,1,4,2017-05-28 23:38:05
1,2,1,5,2017-05-28 23:38:06
2,1,2,1,2017-05-29 23:30:05
3,2,4,3,2017-05-29 23:32:05
4,3,3,2,2017-05-29 22:12:12
5,4,3,1,2017-05-30 22:12:15
6,1,5,2,2017-05-30 22:12:20
7,5,2,3,2017-05-31 23:38:05
8,4,2,4,2017-05-31 23:38:08
9,6,6,4,2017-05-31 23:38:20


,book_id,book_name,avg_rating
0,1,To Kill a MockingBird,5
1,2,Don Quixote,4
2,3,Lazarillo de Tormes,4
3,4,1984,5
4,5,Fifty Shades of Grey,1
5,6,Los hombres que no amaban a las mujeres,4
6,7,Corazon de tinta,5
7,8,Harry Potter,5


1. Obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks. (Para este punto se puede ignorar el segundo DataFrame).

Si queremos hacer un top en pandas segun rating debemos hacer si o si:

1. Filtrar los libros con un mínimo de ratings, para evitar caer en #LaEcuacionMasPeligrosa. Lo que se intenta evitar es que un libro desconocido que solo una persona le haya puesto 5 estrellas este en el top. **Esto esta implicito con el ejercicio, se nos tiene que ocurrir!!**

1. Ordenar por rating **promedio**

1. Evitar tener duplicados

Una forma que tenemos de hacerlo es con el paradigma split apply combine.

1. Agrupamos por libro (split)

1. Filtramos los que tengan la cantidad de registros mayor a x, con x un corte arbitrario (pero con criterio!). **Hacerlo antes que el calculo del promedio nos ayuda a computar menos datos**

1. Hacemos dos columnas nuevas de agregación, una del rating promedio y otra de la cantidad de registros

1. Devolvemos el top 5 ordenado por rating promedio

In [88]:
agrupado = ebooks_df.groupby('book_id')
agrupado.head()

,user_id,book_id,rating,timestamp
0,1,1,4,2017-05-28 23:38:05
1,2,1,5,2017-05-28 23:38:06
2,1,2,1,2017-05-29 23:30:05
3,2,4,3,2017-05-29 23:32:05
4,3,3,2,2017-05-29 22:12:12
5,4,3,1,2017-05-30 22:12:15
6,1,5,2,2017-05-30 22:12:20
7,5,2,3,2017-05-31 23:38:05
8,4,2,4,2017-05-31 23:38:08
9,6,6,4,2017-05-31 23:38:20


In [89]:
CORTE = 1 # Tener en cuenta la magnitud del dataset. Acá hay 15 registros, un corte de 1 esta bien.

ebooks_df_filtrado = agrupado.filter(lambda x: x['rating'].count() > CORTE)
ebooks_df_filtrado

,user_id,book_id,rating,timestamp
0,1,1,4,2017-05-28 23:38:05
1,2,1,5,2017-05-28 23:38:06
2,1,2,1,2017-05-29 23:30:05
3,2,4,3,2017-05-29 23:32:05
4,3,3,2,2017-05-29 22:12:12
5,4,3,1,2017-05-30 22:12:15
7,5,2,3,2017-05-31 23:38:05
8,4,2,4,2017-05-31 23:38:08
10,6,4,4,2017-05-31 23:38:25
11,1,7,4,2017-05-31 23:38:26


In [90]:
# OJO! Un Filter sobre un group by devuelve un dataframe. Hay que agrupar de nuevo!

agrupado_filtrado = ebooks_df_filtrado.groupby('book_id')
promedios = agrupado_filtrado.agg({'rating':'mean'}) # Equivalente a agrupado_filtrado['rating'].mean() o agrupado_filtrado['rating'].agg('mean')
promedios

,rating
book_id,
1,4.500000
2,2.666667
3,1.500000
4,3.500000
7,4.000000
8,4.000000


In [95]:
promedios_ordenado = promedios.sort_values('rating', ascending=False)
promedios_ordenado

,rating
book_id,
1,4.500000
7,4.000000
8,4.000000
4,3.500000
2,2.666667
3,1.500000


In [98]:
# Finalmente, tomamos el top 5
top_5 = promedios_ordenado.head(5)
top_5

,rating
book_id,
1,4.500000
7,4.000000
8,4.000000
4,3.500000
2,2.666667


2. Obtener un DataFrame que indique qué libros tienen una diferencia de rating promedio mayor al 20% entre el sitio de Ebooks y GoodReads.

Acá, vamos a tener que juntar las dos bases de datos, calcular el rating promedio, calcular la el porcentaje de ese rating promedio sobre el promedio de goodreads, filtrar para que ese número sea mayor al 20% y devolver todos los nombres.

Entonces:

1. Hacer un inner join entre ambos datasets, usando la columna de book_id para juntar.

1. Agrupar por libro, y armar una columna que sea el porcentaje de rating promedio de ebooks sobre el rating de goodreads. **Nuevamente filtrar con algun corte para evitar tener libros con pocos ratings buenos**

1. Quedarnos con los que esa columna sea mayor a 20%

1. Devolver sus nombres

In [103]:
df_mergeado = ebooks_df.merge(goodreads_df)
df_mergeado

,user_id,book_id,rating,timestamp,book_name,avg_rating
0,1,1,4,2017-05-28 23:38:05,To Kill a MockingBird,5
1,2,1,5,2017-05-28 23:38:06,To Kill a MockingBird,5
2,1,2,1,2017-05-29 23:30:05,Don Quixote,4
3,5,2,3,2017-05-31 23:38:05,Don Quixote,4
4,4,2,4,2017-05-31 23:38:08,Don Quixote,4
5,2,4,3,2017-05-29 23:32:05,1984,5
6,6,4,4,2017-05-31 23:38:25,1984,5
7,3,3,2,2017-05-29 22:12:12,Lazarillo de Tormes,4
8,4,3,1,2017-05-30 22:12:15,Lazarillo de Tormes,4
9,1,5,2,2017-05-30 22:12:20,Fifty Shades of Grey,1


In [106]:
CORTE = 1 # Tener en cuenta la magnitud del dataset. Acá hay 15 registros, un corte de 1 esta bien.

df_mergeado_filtrado = df_mergeado.groupby('book_id').filter(lambda x: x['rating'].count() > CORTE)
df_mergeado_filtrado

,user_id,book_id,rating,timestamp,book_name,avg_rating
0,1,1,4,2017-05-28 23:38:05,To Kill a MockingBird,5
1,2,1,5,2017-05-28 23:38:06,To Kill a MockingBird,5
2,1,2,1,2017-05-29 23:30:05,Don Quixote,4
3,5,2,3,2017-05-31 23:38:05,Don Quixote,4
4,4,2,4,2017-05-31 23:38:08,Don Quixote,4
5,2,4,3,2017-05-29 23:32:05,1984,5
6,6,4,4,2017-05-31 23:38:25,1984,5
7,3,3,2,2017-05-29 22:12:12,Lazarillo de Tormes,4
8,4,3,1,2017-05-30 22:12:15,Lazarillo de Tormes,4
11,1,7,4,2017-05-31 23:38:26,Corazon de tinta,5


In [156]:
promedios = df_mergeado_filtrado.groupby('book_id').agg({'rating':'mean'})
promedios = promedios.rename(columns={'rating': 'avg_rating_ebook'})
df_con_promedios = df_mergeado_filtrado.merge(promedios, right_index=True, left_on='book_id')
df_con_promedios

,user_id,book_id,rating,timestamp,book_name,avg_rating,avg_rating_ebook
0,1,1,4,2017-05-28 23:38:05,To Kill a MockingBird,5,4.500000
1,2,1,5,2017-05-28 23:38:06,To Kill a MockingBird,5,4.500000
2,1,2,1,2017-05-29 23:30:05,Don Quixote,4,2.666667
3,5,2,3,2017-05-31 23:38:05,Don Quixote,4,2.666667
4,4,2,4,2017-05-31 23:38:08,Don Quixote,4,2.666667
5,2,4,3,2017-05-29 23:32:05,1984,5,3.500000
6,6,4,4,2017-05-31 23:38:25,1984,5,3.500000
7,3,3,2,2017-05-29 22:12:12,Lazarillo de Tormes,4,1.500000
8,4,3,1,2017-05-30 22:12:15,Lazarillo de Tormes,4,1.500000
11,1,7,4,2017-05-31 23:38:26,Corazon de tinta,5,4.000000


In [158]:
df_con_promedios['porcentaje'] = df_con_promedios['avg_rating_ebook'] / df_con_promedios['avg_rating']
mayores_a_20 = df_con_promedios[df_con_promedios['porcentaje']>0.2]

In [159]:
mayores_a_20['book_name'].unique()

array(['To Kill a MockingBird', 'Don Quixote', '1984',
       'Lazarillo de Tormes', 'Corazon de tinta', 'Harry Potter'],
      dtype=object)

Leer:

* https://pandas.pydata.org/pandas-docs/stable/groupby.html

* https://www.datacamp.com/community/tutorials/pandas-split-apply-combine-groupby